In [1]:
!gcloud config set project etsy-bigquery-adhoc-prod

Updated property [core/project].


In [2]:
%load_ext google.cloud.bigquery

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

/home/yzhang/anaconda3/envs/third-pass-nr/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/yzhang/anaconda3/envs/third-pass-nr/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead.

In [6]:
%%bigquery df
with query_listing_with_purchases_clicks as (
    select distinct query, listing_id, purchases, clicks
    from `etsy-data-warehouse-prod.search.visit_level_listing_impressions`
    where _date between date('2023-04-24') and date('2023-06-22')
    and page in ('search', 'market')
    and page_no <= 3
    and purchases > 0
), query_top_purchase_listing as (
    select *
    from query_listing_with_purchases_clicks
    qualify row_number() over (partition by query order by purchases desc) <= 5
)
select * from query_top_purchase_listing
order by query, purchases desc

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████| 5488218/5488218 [00:04<00:00, 1307442.11rows/s]


In [8]:
df.head()

,query,listing_id,purchases,clicks
0,!*i20,1458746513,1,1
1,"""""",1279398268,34,35
2,"""""",1409598243,28,28
3,"""""",1202933984,28,8
4,"""""",1334690513,27,27
